# Flux optique Lucas-Kanade dans OpenCV

### Nous comprendrons les concepts de flux optique et son estimation à l'aide de la méthode de Lucas-Kanade.

#### Objectif 
#### 1-Utilisation de la fonction cv.calcOpticalFlowPyrLK () de la bibliothèque d'openCV pour suivre les points caractéristiques d'une vidéo.
#### 2-Céation d'un champ de flux optique dense apr l'utilisation de la méthode cv.calcOpticalFlowFarneback () .

# 1 -  la fonction cv.calcOpticalFlowPyrLK ()

In [2]:
import numpy as np
import cv2 as cv
from IPython.display import Video
cap = cv.VideoCapture("video/slow_traffic_small.mp4")


In [3]:
#Lecture de video d'endtée

Video("video/slow_traffic_small.mp4"),

(<IPython.core.display.Video object>,)

#### Nous alons déterminer les paramètre pour les coins

In [4]:
# params pour la détection des coins ShiTomasi
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )


#### Paramètres pour le flux optique lucas kanade

In [5]:
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

In [6]:
# Créez des couleurs aléatoires
color = np.random.randint(0,255,(100,3))

# # Prenez la première image et trouvez-y des coins
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

In [8]:
# Créer une image de masque à des fins de dessin
mask = np.zeros_like(old_frame)
i=0
while(1):
        ret,frame=cap.read()
        if (ret==False):
            break
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        # Sélectionnez les bons points
        if p1 is not None:
            good_new = p1[st==1]
            good_old = p0[st==1]
        # dessiner les pistes
        for i,(new,old) in enumerate(zip(good_new, good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
        img = cv.add(frame,mask)
        cv.imshow('frame',img)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
        # Maintenant, mettez à jour l'image précédente et les points précédents
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)
# Release handle to the webcam
cap.release()
cv.destroyAllWindows()

#  Flux optique dense dans OpenCV

In [ ]:
# Flux optique Lucas-Kanade dans OpenCV

In [9]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("video/slow_traffic_small.mp4"))


In [10]:
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
nb=0
j=0
Timg=[][]
while(1):
    ret, frame2 = cap.read()
    if ret==False:
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2',bgr)
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    elif k == ord('s'):
        Timg.appen(frame2,bgr)
        cv.imwrite('opticalfb.png',frame2)
        cv.imwrite('opticalhsv.png',bgr)
    prvs = next
# Release handle to the webcam
cap.release()
cv.destroyAllWindows()